# Week 6: Urban Heat Island Analysis
## Identifying and Analyzing UHI Effects in Seoul

**Instructor**: Sohn Chul

---

## 🎯 Learning Objectives

By the end of this session, you will be able to:
1. Understand the Urban Heat Island (UHI) phenomenon
2. Calculate UHI intensity from sensor data
3. Identify UHI hotspots and cool islands
4. Analyze factors contributing to UHI
5. Assess mitigation strategies

## 1. Setup and Data Loading

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from folium import plugins
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats, spatial
from scipy.interpolate import griddata
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from shapely.geometry import Point, Polygon
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('RdYlBu_r')

print("✅ Libraries imported successfully!")

## 2. Generate Urban Environment Data

In [ ]:
# Create synthetic urban environment data
np.random.seed(42)

# Define Seoul's approximate boundaries
seoul_bounds = {
    'lat_min': 37.45, 'lat_max': 37.65,
    'lon_min': 126.85, 'lon_max': 127.15
}

# Create grid of measurement points
n_sensors = 200

# Define urban zones with different characteristics
urban_zones = [
    {'name': 'CBD', 'center': (37.566, 127.0), 'radius': 0.03, 'intensity': 3.5},
    {'name': 'Gangnam', 'center': (37.517, 127.047), 'radius': 0.025, 'intensity': 3.0},
    {'name': 'Yeouido', 'center': (37.529, 126.917), 'radius': 0.02, 'intensity': 2.8},
    {'name': 'Jamsil', 'center': (37.513, 127.1), 'radius': 0.025, 'intensity': 2.5},
]

# Green spaces (parks) - cooling effect
green_spaces = [
    {'name': 'Namsan Park', 'center': (37.551, 126.991), 'radius': 0.015, 'cooling': -2.5},
    {'name': 'Seoul Forest', 'center': (37.544, 127.038), 'radius': 0.02, 'cooling': -2.0},
    {'name': 'Olympic Park', 'center': (37.521, 127.124), 'radius': 0.025, 'cooling': -2.2},
    {'name': 'Hangang Park', 'center': (37.530, 126.970), 'radius': 0.03, 'cooling': -1.8},
]

# Generate sensor locations
sensor_lats = np.random.uniform(seoul_bounds['lat_min'], seoul_bounds['lat_max'], n_sensors)
sensor_lons = np.random.uniform(seoul_bounds['lon_min'], seoul_bounds['lon_max'], n_sensors)

# Calculate base temperature for each sensor
base_temp = 28 + np.random.normal(0, 1, n_sensors)

# Apply urban heat island effects
uhi_effect = np.zeros(n_sensors)

for i in range(n_sensors):
    lat, lon = sensor_lats[i], sensor_lons[i]
    
    # Add UHI effect from urban zones
    for zone in urban_zones:
        dist = np.sqrt((lat - zone['center'][0])**2 + (lon - zone['center'][1])**2)
        if dist < zone['radius']:
            uhi_effect[i] += zone['intensity'] * (1 - dist/zone['radius'])
    
    # Add cooling effect from green spaces
    for park in green_spaces:
        dist = np.sqrt((lat - park['center'][0])**2 + (lon - park['center'][1])**2)
        if dist < park['radius']:
            uhi_effect[i] += park['cooling'] * (1 - dist/park['radius'])

# Create DataFrame
df_uhi = pd.DataFrame({
    'sensor_id': [f'S{i:04d}' for i in range(n_sensors)],
    'latitude': sensor_lats,
    'longitude': sensor_lons,
    'base_temperature': base_temp,
    'uhi_effect': uhi_effect,
    'temperature': base_temp + uhi_effect,
    'humidity': 65 - uhi_effect * 3 + np.random.normal(0, 5, n_sensors)  # Lower humidity in UHI
})

# Classify land use type based on UHI effect
df_uhi['land_use'] = pd.cut(df_uhi['uhi_effect'], 
                            bins=[-10, -1, 0.5, 2, 10],
                            labels=['Park', 'Residential', 'Commercial', 'Industrial'])

print(f"✅ Created {n_sensors} sensor locations with UHI effects")
print(f"\nTemperature Statistics:")
print(df_uhi[['base_temperature', 'uhi_effect', 'temperature']].describe())

In [ ]:
# KMA Heat Index Calculation Functions
def calculate_wet_bulb_temperature(Ta, RH):
    """Calculate wet-bulb temperature using Stull's formula."""
    Tw = (Ta * np.arctan(0.151977 * (RH + 8.313659)**0.5) + 
          np.arctan(Ta + RH) - 
          np.arctan(RH - 1.67633) + 
          0.00391838 * RH**1.5 * np.arctan(0.023101 * RH) - 
          4.686035)
    return Tw

def calculate_heat_index_kma(Ta, RH):
    """Calculate heat index using KMA (Korea Meteorological Administration) formula."""
    # Step 1: Calculate wet-bulb temperature
    Tw = calculate_wet_bulb_temperature(Ta, RH)
    
    # Step 2: Calculate heat index using KMA formula
    HI = (-0.2442 + 
          0.55399 * Tw + 
          0.45535 * Ta - 
          0.0022 * Tw**2 + 
          0.00278 * Tw * Ta + 
          3.0)
    
    return HI

# Calculate KMA heat index for all sensors
df_uhi['heat_index'] = calculate_heat_index_kma(
    df_uhi['temperature'].values,
    df_uhi['humidity'].values
)

print(f"✅ KMA Heat Index calculated for {n_sensors} sensors")
print(f"\nHeat Index Statistics (KMA Formula):")
print(df_uhi[['temperature', 'humidity', 'heat_index']].describe())

## 3. UHI Intensity Calculation

In [ ]:
# Calculate UHI intensity
# Method 1: Urban-Rural difference
rural_threshold = df_uhi['uhi_effect'].quantile(0.1)  # Bottom 10% as rural reference
rural_sensors = df_uhi[df_uhi['uhi_effect'] <= rural_threshold]
urban_sensors = df_uhi[df_uhi['uhi_effect'] > rural_threshold]

rural_temp_mean = rural_sensors['temperature'].mean()
urban_temp_mean = urban_sensors['temperature'].mean()
uhi_intensity = urban_temp_mean - rural_temp_mean

print(f"📊 UHI Intensity Analysis:")
print(f"  Rural mean temperature: {rural_temp_mean:.2f}°C")
print(f"  Urban mean temperature: {urban_temp_mean:.2f}°C")
print(f"  UHI Intensity: {uhi_intensity:.2f}°C")

# Method 2: Maximum UHI intensity
max_urban_temp = df_uhi['temperature'].max()
min_rural_temp = rural_sensors['temperature'].min()
max_uhi_intensity = max_urban_temp - rural_temp_mean

print(f"\n  Maximum UHI Intensity: {max_uhi_intensity:.2f}°C")
print(f"  Location of max temperature: Lat {df_uhi.loc[df_uhi['temperature'].idxmax(), 'latitude']:.4f}, "
      f"Lon {df_uhi.loc[df_uhi['temperature'].idxmax(), 'longitude']:.4f}")

## 4. Spatial Distribution of UHI

In [ ]:
# Create spatial visualization of UHI
fig, axes = plt.subplots(1, 2, figsize=(15, 7))

# Temperature distribution
scatter1 = axes[0].scatter(df_uhi['longitude'], df_uhi['latitude'], 
                          c=df_uhi['temperature'], s=50, 
                          cmap='RdYlBu_r', alpha=0.7, edgecolors='black', linewidth=0.5)
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
axes[0].set_title('Temperature Distribution', fontweight='bold')
plt.colorbar(scatter1, ax=axes[0], label='Temperature (°C)')

# UHI Effect distribution
scatter2 = axes[1].scatter(df_uhi['longitude'], df_uhi['latitude'], 
                          c=df_uhi['uhi_effect'], s=50, 
                          cmap='RdBu_r', vmin=-3, vmax=3,
                          alpha=0.7, edgecolors='black', linewidth=0.5)
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
axes[1].set_title('UHI Effect Distribution', fontweight='bold')
plt.colorbar(scatter2, ax=axes[1], label='UHI Effect (°C)')

# Add urban zone markers
for zone in urban_zones:
    axes[1].scatter(zone['center'][1], zone['center'][0], 
                   s=200, marker='*', color='red', edgecolor='black', 
                   linewidth=1, label=zone['name'], zorder=5)

# Add green space markers
for park in green_spaces:
    axes[1].scatter(park['center'][1], park['center'][0], 
                   s=200, marker='^', color='green', edgecolor='black', 
                   linewidth=1, label=park['name'], zorder=5)

plt.suptitle('Urban Heat Island Spatial Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Interactive UHI Map

In [ ]:
# Create interactive map with UHI visualization
seoul_center = [37.5665, 126.9780]
m = folium.Map(location=seoul_center, zoom_start=11, tiles='OpenStreetMap')

# Add heat map layer for UHI effect
heat_data = [[row['latitude'], row['longitude'], row['uhi_effect'] + 3]  # Offset for visualization
             for idx, row in df_uhi.iterrows()]

plugins.HeatMap(heat_data, radius=20, blur=15, max_zoom=1).add_to(m)

# Add markers for hotspots and cool islands
for idx, row in df_uhi.iterrows():
    if row['uhi_effect'] > 2:  # Hotspots
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            popup=f"Hotspot: {row['temperature']:.1f}°C\nUHI: +{row['uhi_effect']:.1f}°C",
            color='red',
            fill=True,
            fillColor='red',
            fillOpacity=0.6
        ).add_to(m)
    elif row['uhi_effect'] < -1:  # Cool islands
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            popup=f"Cool Island: {row['temperature']:.1f}°C\nUHI: {row['uhi_effect']:.1f}°C",
            color='blue',
            fill=True,
            fillColor='blue',
            fillOpacity=0.6
        ).add_to(m)

# Add zone boundaries
for zone in urban_zones:
    folium.Circle(
        location=zone['center'],
        radius=zone['radius'] * 111000,  # Convert to meters (approximate)
        popup=f"{zone['name']} (Urban Core)",
        color='red',
        fill=False,
        weight=2
    ).add_to(m)

for park in green_spaces:
    folium.Circle(
        location=park['center'],
        radius=park['radius'] * 111000,
        popup=f"{park['name']} (Green Space)",
        color='green',
        fill=False,
        weight=2
    ).add_to(m)

# Add title
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Seoul Urban Heat Island Map</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_html))

print("✅ Interactive UHI map created")
m

## 6. Land Use Impact Analysis

In [ ]:
# Analyze temperature by land use type
land_use_stats = df_uhi.groupby('land_use').agg({
    'temperature': ['mean', 'std', 'min', 'max'],
    'uhi_effect': 'mean',
    'humidity': 'mean',
    'sensor_id': 'count'
}).round(2)

land_use_stats.columns = ['Temp Mean', 'Temp Std', 'Temp Min', 'Temp Max', 
                          'UHI Effect', 'Humidity', 'Count']

print("📊 Temperature by Land Use Type:")
print(land_use_stats)

# Visualize land use impact
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Temperature by land use
land_use_order = ['Park', 'Residential', 'Commercial', 'Industrial']
colors = ['green', 'yellow', 'orange', 'red']

for i, lu in enumerate(land_use_order):
    if lu in df_uhi['land_use'].values:
        data = df_uhi[df_uhi['land_use'] == lu]['temperature']
        axes[0].boxplot(data, positions=[i], widths=0.6, 
                       patch_artist=True, 
                       boxprops=dict(facecolor=colors[i], alpha=0.7))

axes[0].set_xticklabels(land_use_order)
axes[0].set_ylabel('Temperature (°C)')
axes[0].set_title('Temperature Distribution by Land Use')
axes[0].grid(True, alpha=0.3)

# UHI effect by land use
uhi_means = [df_uhi[df_uhi['land_use'] == lu]['uhi_effect'].mean() 
             if lu in df_uhi['land_use'].values else 0 
             for lu in land_use_order]
bars = axes[1].bar(land_use_order, uhi_means, color=colors, alpha=0.7)
axes[1].set_ylabel('UHI Effect (°C)')
axes[1].set_title('Average UHI Effect by Land Use')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1].grid(True, alpha=0.3)

# Humidity by land use
humidity_means = [df_uhi[df_uhi['land_use'] == lu]['humidity'].mean() 
                 if lu in df_uhi['land_use'].values else 0 
                 for lu in land_use_order]
axes[2].bar(land_use_order, humidity_means, color=colors, alpha=0.7)
axes[2].set_ylabel('Humidity (%)')
axes[2].set_title('Average Humidity by Land Use')
axes[2].grid(True, alpha=0.3)

plt.suptitle('Land Use Impact on Urban Climate', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Temporal UHI Patterns

In [ ]:
# Simulate temporal UHI patterns (24 hours)
hours = np.arange(24)

# Different temporal patterns for urban and rural areas
# Urban areas retain heat longer
urban_pattern = 28 + 6 * np.sin((hours - 6) * np.pi / 12) + np.random.normal(0, 0.5, 24)
rural_pattern = 26 + 8 * np.sin((hours - 6) * np.pi / 12) + np.random.normal(0, 0.5, 24)

# UHI intensity throughout the day
uhi_temporal = urban_pattern - rural_pattern

# Create temporal visualization
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Temperature Patterns', 'UHI Intensity'),
    shared_xaxes=True,
    vertical_spacing=0.1
)

# Temperature patterns
fig.add_trace(
    go.Scatter(x=hours, y=urban_pattern, mode='lines+markers',
               name='Urban', line=dict(color='red', width=2)),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=hours, y=rural_pattern, mode='lines+markers',
               name='Rural', line=dict(color='green', width=2)),
    row=1, col=1
)

# UHI intensity
fig.add_trace(
    go.Scatter(x=hours, y=uhi_temporal, mode='lines+markers',
               name='UHI Intensity', line=dict(color='orange', width=2),
               fill='tozeroy', fillcolor='rgba(255,165,0,0.2)'),
    row=2, col=1
)

fig.update_xaxes(title_text="Hour of Day", row=2, col=1)
fig.update_yaxes(title_text="Temperature (°C)", row=1, col=1)
fig.update_yaxes(title_text="UHI Intensity (°C)", row=2, col=1)

fig.update_layout(height=600, title_text="Temporal UHI Patterns", showlegend=True)
fig.show()

# Peak UHI times
peak_uhi_hour = hours[np.argmax(uhi_temporal)]
min_uhi_hour = hours[np.argmin(uhi_temporal)]

print(f"\n⏰ Temporal UHI Analysis:")
print(f"  Peak UHI intensity: {uhi_temporal.max():.2f}°C at {peak_uhi_hour:02d}:00")
print(f"  Minimum UHI intensity: {uhi_temporal.min():.2f}°C at {min_uhi_hour:02d}:00")
print(f"  Daily average UHI: {uhi_temporal.mean():.2f}°C")

## 8. UHI Clustering Analysis

In [ ]:
# Perform clustering to identify UHI zones
from sklearn.cluster import KMeans

# Prepare features for clustering
features = df_uhi[['latitude', 'longitude', 'temperature', 'uhi_effect']].values
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Perform K-means clustering
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df_uhi['cluster'] = kmeans.fit_predict(features_scaled)

# Analyze clusters
cluster_stats = df_uhi.groupby('cluster').agg({
    'temperature': 'mean',
    'uhi_effect': 'mean',
    'humidity': 'mean',
    'sensor_id': 'count'
}).round(2)

cluster_stats.columns = ['Avg Temp', 'Avg UHI', 'Avg Humidity', 'Count']
cluster_stats = cluster_stats.sort_values('Avg UHI', ascending=False)

# Assign cluster labels based on UHI intensity
cluster_labels = ['Strong UHI', 'Moderate UHI', 'Mild UHI', 'Neutral', 'Cool Island']
cluster_mapping = dict(zip(cluster_stats.index, cluster_labels))
df_uhi['uhi_zone'] = df_uhi['cluster'].map(cluster_mapping)

print("📊 UHI Zone Classification:")
print(cluster_stats)

# Visualize clusters
fig, ax = plt.subplots(figsize=(12, 8))

colors = ['darkred', 'red', 'orange', 'yellow', 'blue']
for i, (cluster_id, label) in enumerate(cluster_mapping.items()):
    mask = df_uhi['cluster'] == cluster_id
    ax.scatter(df_uhi[mask]['longitude'], df_uhi[mask]['latitude'],
              c=colors[i], label=label, s=50, alpha=0.7, edgecolors='black', linewidth=0.5)

ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)
ax.set_title('UHI Zone Classification', fontsize=14, fontweight='bold')
ax.legend(title='UHI Zone')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Mitigation Strategy Assessment

In [ ]:
# Simulate mitigation strategies
mitigation_strategies = {
    'Green Roof': {'cooling': -1.5, 'cost': 100, 'applicability': 0.3},
    'Cool Pavement': {'cooling': -1.0, 'cost': 50, 'applicability': 0.5},
    'Urban Forest': {'cooling': -2.0, 'cost': 75, 'applicability': 0.2},
    'Water Features': {'cooling': -1.2, 'cost': 150, 'applicability': 0.1},
    'Building Shade': {'cooling': -0.8, 'cost': 30, 'applicability': 0.4}
}

# Calculate potential impact for each strategy
hotspot_sensors = df_uhi[df_uhi['uhi_effect'] > 2]
n_hotspots = len(hotspot_sensors)

print("🌿 Mitigation Strategy Assessment:")
print(f"\nNumber of hotspot sensors to treat: {n_hotspots}")
print("\nStrategy Analysis:")
print("-" * 70)

strategy_impact = []

for strategy, params in mitigation_strategies.items():
    applicable_sensors = int(n_hotspots * params['applicability'])
    total_cooling = applicable_sensors * params['cooling']
    total_cost = applicable_sensors * params['cost']
    cost_effectiveness = abs(total_cooling) / total_cost if total_cost > 0 else 0
    
    strategy_impact.append({
        'Strategy': strategy,
        'Cooling Effect (°C)': params['cooling'],
        'Applicable Sensors': applicable_sensors,
        'Total Cooling': total_cooling,
        'Total Cost': total_cost,
        'Cost Effectiveness': cost_effectiveness
    })
    
    print(f"{strategy:15s}: Cooling={params['cooling']:5.1f}°C, "
          f"Applicable={applicable_sensors:3d}, "
          f"Total Effect={total_cooling:6.1f}°C, "
          f"Cost=${total_cost:5d}")

# Convert to DataFrame for visualization
df_strategies = pd.DataFrame(strategy_impact)

# Visualize strategy comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Cooling effect vs cost
for _, row in df_strategies.iterrows():
    axes[0].scatter(row['Total Cost'], abs(row['Total Cooling']), 
                   s=200, alpha=0.7, edgecolors='black', linewidth=1)
    axes[0].annotate(row['Strategy'], 
                    (row['Total Cost'], abs(row['Total Cooling'])),
                    xytext=(5, 5), textcoords='offset points', fontsize=9)

axes[0].set_xlabel('Total Cost ($)', fontsize=12)
axes[0].set_ylabel('Total Cooling Effect (°C)', fontsize=12)
axes[0].set_title('Mitigation Strategy Cost-Benefit Analysis', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Cost effectiveness
strategies = df_strategies['Strategy'].values
effectiveness = df_strategies['Cost Effectiveness'].values
bars = axes[1].barh(strategies, effectiveness, color='teal', alpha=0.7)
axes[1].set_xlabel('Cost Effectiveness (°C per $)', fontsize=12)
axes[1].set_title('Strategy Cost Effectiveness', fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.suptitle('UHI Mitigation Strategy Assessment', fontsize=16)
plt.tight_layout()
plt.show()

## 10. Summary Report

In [ ]:
# Generate comprehensive UHI analysis report
report = f"""
URBAN HEAT ISLAND ANALYSIS REPORT
==================================

OVERALL UHI CHARACTERISTICS:
- Average UHI Intensity: {uhi_intensity:.2f}°C
- Maximum UHI Intensity: {max_uhi_intensity:.2f}°C
- Number of Hotspots (>2°C): {len(df_uhi[df_uhi['uhi_effect'] > 2])}
- Number of Cool Islands (<-1°C): {len(df_uhi[df_uhi['uhi_effect'] < -1])}

SPATIAL DISTRIBUTION:
- Strongest UHI zones: Industrial and Commercial areas
- Cooling zones: Parks and green spaces
- Coverage: {len(df_uhi)} measurement points

TEMPORAL PATTERNS:
- Peak UHI: {peak_uhi_hour:02d}:00 ({uhi_temporal[peak_uhi_hour]:.2f}°C)
- Minimum UHI: {min_uhi_hour:02d}:00 ({uhi_temporal[min_uhi_hour]:.2f}°C)
- Daily Average: {uhi_temporal.mean():.2f}°C

LAND USE IMPACT:
{land_use_stats.to_string()}

RECOMMENDED MITIGATION STRATEGIES:
1. Urban Forest: Highest cooling potential (-2.0°C per location)
2. Green Roof: Good for dense urban areas (-1.5°C)
3. Cool Pavement: Wide applicability (-1.0°C)

PRIORITY AREAS FOR INTERVENTION:
- {len(hotspot_sensors)} sensors in hotspot zones
- Focus on industrial and commercial districts
- Enhance green infrastructure in dense urban cores
"""

print(report)

# Save results
df_uhi.to_csv('../data/processed/uhi_analysis_results.csv', index=False)
print("\n✅ UHI analysis results saved")

## 11. Assignment

### Week 6 Tasks:

1. **UHI Calculation** (25 points)
   - Calculate UHI intensity using multiple methods
   - Compare urban-rural temperature differences
   - Identify maximum UHI locations

2. **Spatial Analysis** (25 points)
   - Map UHI distribution across the city
   - Identify hotspots and cool islands
   - Perform spatial clustering analysis

3. **Contributing Factors** (25 points)
   - Analyze land use impact on UHI
   - Investigate temporal patterns
   - Assess environmental correlations

4. **Mitigation Assessment** (25 points)
   - Evaluate mitigation strategies
   - Perform cost-benefit analysis
   - Recommend priority interventions

### Bonus Challenge:
- Integrate satellite imagery for validation
- Model UHI under climate change scenarios
- Design optimal green infrastructure network

## Summary

In this week, we covered:
- ✅ Urban Heat Island concept and measurement
- ✅ UHI intensity calculation methods
- ✅ Spatial and temporal UHI patterns
- ✅ Land use impact analysis
- ✅ Mitigation strategy assessment

### Next Week Preview:
**Week 7: Statistical Modeling**
- Regression analysis for heat prediction
- Time series forecasting
- Correlation and causation analysis
- Model validation techniques

### Resources:
- [Urban Heat Island EPA Guide](https://www.epa.gov/heatislands)
- [UHI Mitigation Strategies](https://www.sciencedirect.com/topics/engineering/urban-heat-island-mitigation)
- [Remote Sensing of UHI](https://www.mdpi.com/2072-4292/11/1/48)

---
**End of Week 6**

*Instructor: Sohn Chul*